<div style="display: flex; align-items: center; padding: 10px;">
  <a href="https://www.utem.cl/" target="_blank" style="display: inline-block; text-decoration: none;">
    <img src="https://fing.utem.cl/wp-content/themes/fing_21082024/images/08_img_fac_ing/00_id_corp/imagotipo_utem.png" alt="Visit UTEM" width="243" style="vertical-align: middle;"/>
  </a>


<style>
  a img {
    border: none;
    outline: none;
  }
</style>

<div>
  <a href="https://learn.nvidia.com" target="_blank" style="display: inline-block; text-decoration: none;">
    <img src="https://aidl.uniwa.gr/wp-content/uploads/2021/02/nvidia-300x113.png" alt="Visit NVIDIA" width="150" style="vertical-align: middle;"/>
  </a>
</div>

# Análisis y Visualización de Redes Sociales: Construcción de un Grafo de Interacciones en Reddit Utilizando NetworkX y Aceleración con RAPIDS cuGraph de NVIDIA

Este proyecto examina la estructura y las conexiones de la red social Reddit, específicamente el subreddit r/chile, mediante un análisis de redes sociales. Empezando por la recopilación de datos de conexiones entre usuarios, se crea una matriz de adyacencia que representa estas relaciones. Utilizando NetworkX y la aceleración con cuGraph, exploramos diversas propiedades estructurales del grafo, como el grado de los nodos, centralidad y otras metricas importantes. Este análisis se enfoca en acercar la comprensión de la dinámica de interacción en esta comunidad virtual y facilitar una visualización gráfica para una representación clara de la red.

# Introducción #

El análisis de redes sociales permite obtener una comprensión profunda sobre cómo interactúan los individuos dentro de una comunidad virtual. Reddit, como una de las plataformas sociales más populares, ofrece una vasta cantidad de datos para estudiar patrones de interacción entre usuarios y temas. Este proyecto se centra en la construcción de un grafo que represente estas interacciones en Reddit, enfocándose en un subreddit específico chileno r/Chile. El objetivo es que a partir de estos analisis visualizar mediante el uso de herramientas aceleradas como implementar un grafo con cantidades de datos masivos, para luego entender como estas se relacionan con las matrices de adyacencia con sus respectivas propiedades. Para este fin, se emplea la biblioteca NetworkX, ampliamente utilizada para el análisis de redes complejas, junto con cuGraph, una herramienta de la suite RAPIDS de NVIDIA que acelera el procesamiento gráfico mediante GPUs. El uso de cuGraph en conjunto con NetworkX permite mejorar el rendimiento en la construcción y manipulación del grafo, especialmente útil en datasets extensos. Este enfoque optimizado resulta crucial para el análisis en tiempo real y la visualización de grandes redes, proporcionando así una visión clara de las interacciones en Reddit.


# Procedimiento

**Recopilación de Datos:**

Utilizando la API de Reddit mediante la librería PRAW, se extraen datos relevantes de publicaciones y comentarios en el subreddit seleccionado (en este caso, r/Chile). Esta etapa incluye la recopilación de identificadores de publicaciones, nombres de autores, y conexiones a través de comentarios, lo cual permite crear un conjunto de datos detallado sobre las interacciones entre usuarios y temas en la comunidad. Debido a las restricciones inherentes al uso de entornos con recursos limitados, como GPUs o TPUs T4 en Google Colab, y considerando que el análisis se realiza con fines académicos, se ha ajustado la recopilación de datos para garantizar un enfoque práctico y manejable. Como resultado, la cantidad de nodos y aristas en la red puede fluctuar dependiendo de las características de la muestra seleccionada.

**Construcción del Grafo de Interacciones:**

Con NetworkX, se establece un grafo donde los nodos representan a los usuarios y temas, y las aristas representan las interacciones (como comentarios o respuestas) entre ellos. Este grafo inicial se construye de manera que se capturen las relaciones principales de interés, estableciendo una base para el análisis estructural posterior.

**Transformación y Optimización del Grafo con cuGraph:**

Se transfiere el grafo construido en NetworkX a cuGraph para aprovechar la aceleración de procesamiento en GPU. En esta etapa, se optimizan tareas intensivas de cómputo, como el cálculo de métricas de centralidad de intermediación (para identificar usuarios clave en la red) y la detección de comunidades (para ver grupos de interacción frecuentes o comunidades activas dentro de la red).

**Construcción de la Matriz de Adyacencia:**

Utilizando la estructura del grafo, se construye una matriz de adyacencia que representa las conexiones directas entre nodos. Esta matriz permite analizar la conectividad y estructura de la red en un formato matricial, facilitando cálculos adicionales y sirviendo como base para la visualización en forma de mapa de calor.

**Análisis de Propiedades de la Red:**

Con la matriz de adyacencia y el grafo optimizado en cuGraph, se calculan propiedades clave de la red. Estos análisis incluyen el grado de los nodos, la identificación de nodos centrales mediante métricas como la centralidad de grado, cercanía e intermediación, y la evaluación del PageRank para medir la influencia relativa de cada usuario en la red. Además, se aplica el algoritmo de Louvain para la detección de comunidades, proporcionando información sobre la agrupación y modularidad de la red. También se analizan las componentes conexas para identificar subconjuntos independientes de nodos, y se calcula la densidad de la red, permitiendo evaluar su nivel de interconectividad. Finalmente, se determinan los caminos mínimos entre nodos, lo que ayuda a entender la eficiencia en la transmisión de información dentro de la red.

**Visualización de la Red:**

Para una representación visual clara, se crea una visualización gráfica del grafo utilizando NetworkX, Pyvis y Plotly. Se emplean distintas herramientas de visualización para destacar las conexiones de alta interacción, y se genera un mapa de calor interactivo que muestra la matriz de adyacencia. Esto permite una comprensión visual de las interacciones y la estructura de la red.


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install nx-cugraph-cu12 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 535 (delta 174), reused 130 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (535/535), 172.47 KiB | 883.00 KiB/s, done.
Resolving deltas: 100% (276/276), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.4 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install praw networkx matplotlib pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.8 MB/s eta 0:00:00


In [ ]:
import praw
import cudf
import cugraph as cg
import networkx as nx
import matplotlib.pyplot as plt
import pyvis
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
print(f"using networkx version {nx.__version__}")

nx.config.warnings_to_ignore.add("cache")

using networkx version 3.4.2


In [ ]:
#API de Reddit
reddit = praw.Reddit(
    client_id='YJrvZJGskTXAz44YrYjV9w',
    client_secret='3VAedP_bPoyKioG7mvFeOYw3Pvgwbw',
        user_agent='Useful_Citron5796'
    )

In [ ]:
def build_reddit_graph_cugraph(subreddit_name, limit_posts=100, limit_comments=50):
    edge_list = []
    subreddit = reddit.subreddit(subreddit_name)

    for submission in subreddit.new(limit=limit_posts):
        # Limitador de carga
        submission.comments.replace_more(limit=0)
        post_id = f"post_{submission.id}"
        edge_list.append((subreddit_name, post_id))

        # Agregar el autor del post
        author = submission.author.name if submission.author else "deleted"
        edge_list.append((post_id, author))

        # Agregar los comentarios del post
        for comment in submission.comments[:limit_comments]:
            commenter = comment.author.name if comment.author else "deleted"
            edge_list.append((post_id, commenter))

    # Convertir la lista de aristas a un DataFrame de cuDF
    df = cudf.DataFrame(edge_list, columns=["src", "dst"])

    # Crear el grafo en cuGraph
    g = cg.Graph()
    g.from_cudf_edgelist(df, source="src", destination="dst")
    return g

In [ ]:
async def main():
    reddit = await setup_reddit()
    reddit_graph = await build_reddit_graph_cugraph(reddit, "r/chile", limit_posts=20, limit_comments=10)
    reddit_graph_nx = reddit_graph.to_networkx()

In [ ]:
# Construcción del grafo
reddit_graph = build_reddit_graph_cugraph("chile", limit_posts=200, limit_comments=100)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
# Centralidad de intermediación
bc_df = cg.betweenness_centrality(reddit_graph)
centrality_df = centrality_df.sort_values(by='betweenness_centrality', ascending=False)
print("Betweenness Centrality:")
print(centrality_df.head())

NameError: name 'centrality_df' is not defined

In [ ]:
# Detección de comunidades con Louvain
communities, modularity = cg.louvain(reddit_graph)
print("Modularidad:", modularity)
print(communities.head())

In [ ]:
# Calcular PageRank
pagerank_df = cg.pagerank(reddit_graph)
pagerank_df = pagerank_df.sort_values(by='pagerank', ascending=False)
print("PageRank:")
print(pagerank_df.head())

In [ ]:
# Obtener el número de vértices y aristas
num_vertices = reddit_graph.number_of_vertices()
num_edges = reddit_graph.number_of_edges()

# Calcular la densidad
density = (2 * num_edges) / (num_vertices * (num_vertices - 1))

# Mostrar la densidad
print("Densidad del grafo:", density)

In [ ]:
# Convertir el grafo de cuGraph a NetworkX manualmente
# Extraer las aristas de cuGraph y construir el grafo en NetworkX
edges_df = reddit_graph.view_edge_list().to_pandas()
reddit_graph_nx = nx.from_pandas_edgelist(edges_df, source="src", target="dst")

In [ ]:
# Centralidad de intermediación
bc_df = cg.betweenness_centrality(reddit_graph)

# Detección de comunidades con Louvain
louvain_df, modularity = cg.louvain(reddit_graph)

# PageRank
pagerank_df = cg.pagerank(reddit_graph)
# Convertir métricas a diccionarios
louvain_dict = dict(zip(louvain_df["vertex"].to_pandas(), louvain_df["partition"].to_pandas()))
bc_dict = dict(zip(bc_df["vertex"].to_pandas(), bc_df["betweenness_centrality"].to_pandas()))
pagerank_dict = dict(zip(pagerank_df["vertex"].to_pandas(), pagerank_df["pagerank"].to_pandas()))
# Normalización segura
max_bc = max(bc_dict.values()) if bc_dict else 1
max_pagerank = max(pagerank_dict.values()) if pagerank_dict else 1


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

# Calcular el grado de cada nodo
node_degrees = dict(reddit_graph_nx.degree())
max_degree = max(node_degrees.values())  # Máximo grado para normalización

# Crear una lista de colores para los nodos según el grado (intensidad de interacciones)
colors = [plt.cm.coolwarm(node_degrees[node] / max_degree) for node in reddit_graph_nx.nodes()]

# Dibujar el grafo con los colores basados en el grado de los nodos
plt.figure(figsize=(75, 75))
pos = nx.spring_layout(reddit_graph_nx, k=0.2, iterations=30)

nx.draw(
    reddit_graph_nx, pos,
    with_labels=True,
    node_color=colors,  # Colores según la intensidad de interacción
    edge_color="gray",
    node_size=30, font_size=8
)

plt.title("Visualización del Grafo de Reddit r/chile")
plt.show()

In [ ]:
from pyvis.network import Network

print("Generando visualización interactiva con Pyvis...")

# Crear el grafo en Pyvis
net = Network(notebook=True, cdn_resources="in_line", height="1500px", width="100%")
pos = nx.spring_layout(reddit_graph_nx, k=0.2, iterations=30)
# Agregar nodos al grafo interactivo
for node in reddit_graph_nx.nodes():
    community = louvain_dict.get(node, 0)  # Comunidad Louvain
    bc = bc_dict.get(node, 0) / max_bc  # Normalizar Betweenness
    pagerank = pagerank_dict.get(node, 0) / max_pagerank  # Normalizar PageRank

    # Asignar color por comunidad Louvain
    color = plt.cm.tab20(community % 20)  # Escalar colores hasta 20 comunidades
    color_hex = f'#{int(color[0]*255):02x}{int(color[1]*255):02x}{int(color[2]*255):02x}'

    # Tamaño proporcional al PageRank
    size = 5 + pagerank * 20

    # Agregar nodo al grafo interactivo
    net.add_node(
        node,
        label=node,
        color=color_hex,
        size=size,
        title=f"Betweenness: {bc:.4f}//PageRank: {pagerank:.4f}//Louvain {community:.4f}"
    )

# Agregar aristas al grafo
for edge in reddit_graph_nx.edges():
    net.add_edge(*edge)

# Agregar botones
net.show_buttons()
net.show("reddit_graph_with_metrics.html")
print("Visualización guardada como reddit_graph_with_metrics.html")
# Descargar el archivo en Google Colab
from google.colab import files
files.download("reddit_graph_with_metrics.html")


In [ ]:
# Crear y visualizar la matriz de adyacencia
adj_matrix = nx.to_pandas_adjacency(reddit_graph_nx)
print("Matriz de Adyacencia:")
print(adj_matrix)

In [ ]:
# Grados de cada nodo
node_degrees = dict(reddit_graph_nx.degree())
print(node_degrees)

In [ ]:
# Crear la matriz de adyacencia
adj_matrix = nx.to_pandas_adjacency(reddit_graph_nx)

# Crear una figura de heatmap con Plotly
fig = go.Figure(data=go.Heatmap(
        z=adj_matrix.values,
        x=adj_matrix.columns,
        y=adj_matrix.index,
        colorscale="Viridis"
    ))

# Configuración de la figura
fig.update_layout(
    title="Matriz de Adyacencia Interactiva",
    xaxis_title="Nodo",
    yaxis_title="Nodo",
    xaxis_nticks=36,
    yaxis_nticks=36,
    height=800,
    width=800
)

fig.show()


In [ ]:
# Descargar la matriz de adyacencia como CSV
adj_matrix.to_csv("adjacency_matrix.csv", index=True)
files.download("adjacency_matrix.csv")

# Conclusión
La integración de NetworkX con la computación acelerada de cuGraph ha permitido crear un modelo eficiente y escalable de las interacciones en Reddit, específicamente en el subreddit r/Chile. A través de la matriz de adyacencia, se analizaron propiedades como el grado de los nodos y las componentes conexas, revelando patrones importantes de conexión y comunicación entre los usuarios. La visualización de estos datos ha facilitado una comprensión más profunda de las relaciones en esta red social, destacando la utilidad de las herramientas de análisis de redes para estudiar dinámicas en comunidades virtuales.

---
<i>
For academic use only.
<i>
<br>
<br>
<i>
<br>Authors: Martin Velasco, Ignacio Ramírez, Amelia Rodriguez
<br>Title: Social Media Analysis and Visualization: Building a Reddit Interaction Graph Using NetworkX and Acceleration with NVIDIA RAPIDS cuGraph
<br>Institution: <a href="https://www.utem.cl" target="_blank">Universidad Tecnológica Metropolitana (UTEM)</a>
<br>Github: <a href="https://github.com/altairBASIC" target="_blank">https://github.com/altairBASIC</a>
<br>Date: November 2024

